# **Лабораторная работа №7**
**Чаплин Виталий (гр.5130901/10202)**

# Упражнение 7.2

В этой главе показано, как выразить ДПФ и обратное ДПФ произведениями
матриц. Время выполнения этих операций пропорционально
N^2, где N - длина массива сигнала. Во многих случаях этого
достаточно, но есть алгоритм и побыстрее - быстрое преобразование
Фурье (БПФ); время его работы пропорционально Nlog N.
Ключ к БПФ - это лемма Дэниелсона-Ланцоша (Danielson-
Lanczos).
Эта лемма предлагает рекурсивный алгоритм для ДПФ:
1. Дан массив сигнала у. Разделим его на четные элементы е и
нечетные элементы о.
2. Вычислим DFT е и о, делая рекурсивные вызовы.
3. Вычислим DFT(y) для каждого значения n, используя лемму
Дэниелсона-Ланцоша.
В простейшем случае эту рекурсию надо продолжать, пока длина у
не дойдет до 1. Тогда DFT(y) = у. А если длина у достаточно мала,
можно вычислить его ДПФ перемножением матриц, используя заранее
вычисленные матрицы.

In [1]:
import os

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py
import numpy as np
import matplotlib.pyplot as plt
from thinkdsp import decorate
import numpy as np
PI2 = 2 * np.pi

--2024-04-15 14:44:44--  https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py [following]
--2024-04-15 14:44:44--  https://raw.githubusercontent.com/AllenDowney/ThinkDSP/master/code/thinkdsp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48574 (47K) [text/plain]
Saving to: ‘thinkdsp.py’

thinkdsp.py         100%[===================>]  47.44K  --.-KB/s    in 0.01s   

2024-04-15 14:44:44 (4.40 MB/s) - ‘thinkdsp.py’ saved [48574/48574]



Вычислим БПФ сигнала:

In [8]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Создадим функцию для вычисления DFT:

In [9]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [10]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Теперь создадим функцию, которая делит массив пополам и вычисляет БПФ его половин

In [4]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

 Получаем такие же результаты:

In [5]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Теперь заменим функцию np.fft.fft на рекурсивные вызовы и протестируем новую функцию вычисления БПФ

In [6]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys

    He = fft(ys[::2])
    Ho = fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

 Получаем такие же результаты:

In [7]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Такая реализация БПФ требует времени, пропорционального n log(n). Данное преобразование также занимает пространство, пропорциональное n log(n) и отнимает некоторое время на создание и копирование массивов.